# 구글 드라이브에 마운트

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

# 데이터 호출

In [ ]:
import pandas as pd

df = pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/WMI_Wafer_DefectPattern_Detection_AnomalyAnalysis/data/LSWMD.pkl")
print(df.shape)
print(df["failureType"].head())      # 보통 list 형태거나 빈 list
print(df["trianTestLabel"].head())   # 기존 train/test 표기가 있을 수 있음(우리는 재분할 권장)

# 데이터 구조 한 번 더 확인

In [ ]:
print(df.columns)
print(df[["waferMap","failureType","trianTestLabel"]].head(3))

# waferMap 한 개의 형태 확인
wm0 = df["waferMap"].iloc[0]
print(type(wm0), getattr(wm0, "shape", None))

# 라벨 컬럼 만들기 (failureType가 list인 경우 처리)

In [ ]:
import numpy as np

def get_label(x):
    if isinstance(x, (list, tuple, np.ndarray)):
        return str(x[0][0]) if len(x) > 0 else None
    if isinstance(x, str) and x.strip() != "":
        return x
    return None

df["label"] = df["failureType"].apply(get_label)

print(df["label"].value_counts(dropna=False).head(15))
print("labeled count:", df["label"].notna().sum())

In [ ]:
print(type(df.iloc[0]["label"]))

# 라벨 있는 데이터만 필터링 + Unknown 클래스 홀드아웃 만들기
Donut과 Scratch를 Unknown으로 만들고 진행

In [ ]:
UNKNOWN_CLASSES = ["Donut", "Scratch"]

labeled = df[df["label"].notna()].copy()
known_df = labeled[~labeled["label"].isin(UNKNOWN_CLASSES)].copy()
unknown_df = labeled[labeled["label"].isin(UNKNOWN_CLASSES)].copy()

print("labeled:", len(labeled))
print("known:", len(known_df))
print("unknown:", len(unknown_df))
print("known classes:", sorted(known_df["label"].unique()))
print("unknown classes:", sorted(unknown_df["label"].unique()))

### 클래스 별 대표 1장씩 그리드로 보기

## 각 클래스 별로 그리드로 보기

- **Center**: 웨이퍼 **중심(가운데)**에 불량이 몰려 있는 패턴 (가운데 뭉침)
- **Donut**: 중심은 비교적 깨끗하고, 그 주변에 **동심원(도넛 링)**처럼 불량이 분포 (원형 띠)
- **Edge-Loc (Edge-Local)**: 웨이퍼 **가장자리의 특정 구간**에만 불량이 몰린 패턴 (테두리 한쪽만)
- **Edge-Ring**: 웨이퍼 **테두리 전체를 따라 고리(링)**처럼 불량이 나타나는 패턴 (둘레 전체 띠)
- **Loc (Local)**: 웨이퍼 내부에서 **중심/테두리가 아닌 임의 위치**에 불량이 뭉친 패턴 (안쪽 어딘가 덩어리)
- **Random**: 웨이퍼 전체에 불량이 **듬성듬성, 비교적 고르게 흩어진** 패턴 (여기저기 점)
- **Scratch**: **직선/곡선 형태로 길게 이어진** 불량 패턴 (긁힌 자국처럼 선/호)
- **Near-full**: 웨이퍼 **거의 전체가 불량으로 채워진** 패턴 (대부분 불량)
- **none**: 뚜렷한 결함 패턴이 없거나 결함이 거의 없는 상태 (패턴 없음)
---
- 보라(가장 어두운 색) = 작은 값 (보통 0) → 웨이퍼 바깥/배경
- 파랑(중간 값) = 중간 값 (보통 1) → 정상 die (pass)
- 노랑(가장 밝은 색) = 큰 값 (보통 2) → 불량 die (fail)

In [ ]:
import matplotlib.pyplot as plt

labels = sorted(labeled["label"].unique())
n = len(labels)

cols = 4
rows = (n + cols - 1) // cols

plt.figure(figsize=(4*cols, 4*rows))
for i, lab in enumerate(labels):
    wm = labeled[labeled["label"] == lab]["waferMap"].iloc[0]  # 첫 샘플
    ax = plt.subplot(rows, cols, i+1)
    ax.imshow(wm, interpolation="nearest")
    ax.set_title(lab)
    ax.axis("off")
plt.tight_layout()
plt.show()

### 특정 Failure Type에서 여러 장 랜덤 샘플 보기

In [ ]:
import matplotlib.pyplot as plt

def show_samples(label, n=8, seed=42):
    sub = labeled[labeled["label"] == label]
    if len(sub) == 0:
        print("No samples for:", label)
        return
    sub = sub.sample(n=min(n, len(sub)), random_state=seed)

    cols = 4
    rows = (len(sub) + cols - 1) // cols
    plt.figure(figsize=(4*cols, 4*rows))
    for i, wm in enumerate(sub["waferMap"].tolist()):
        ax = plt.subplot(rows, cols, i+1)
        ax.imshow(wm, interpolation="nearest")
        ax.axis("off")
    plt.suptitle(f"{label} (n={len(sub)})", y=1.02, fontsize=14)
    plt.tight_layout()
    plt.show()

# 예시
for label in ["Center", "Donut", "Edge-Loc", "Edge-Ring", "Loc", "Random", "Scratch", "Near-full", "none"]:
    show_samples(label, n=8, seed=0)

# Train/Val/Test 분할 (Known만 stratify로 분할)
- train = 70%
- val = 15%
- test_known = 15%
- test_unknown = unknown_df 전부

이 비율은 머신러닝에서 아주 흔한 기본값

- train을 충분히 크게 가져가야 학습이 잘 되고
- val/test도 너무 작으면 지표가 흔들리니까 적당히 확보하는 균형

In [ ]:
from sklearn.model_selection import train_test_split

train_df, temp_df = train_test_split(
    known_df, test_size=0.3, random_state=42, stratify=known_df["label"] # stratify는 각 split마다 라벨 비율이 원래 데이터랑 비슷하게 유지되도록 나눠주는 옵션
)
val_df, test_known_df = train_test_split(
    temp_df, test_size=0.5, random_state=42, stratify=temp_df["label"]
)

test_unknown_df = unknown_df.copy()

print("train:", len(train_df), "val:", len(val_df),
      "test_known:", len(test_known_df), "test_unknown:", len(test_unknown_df))

# PyTorch Dataset/DataLoader 만들기 (MVP용 간단 전처리)

*   waferMap을 64×64로 resize
*   1채널을 3채널로 복제해서 ResNet에 넣기 (가장 쉬운 방식)
*   회전/반전 증강은 train에만



In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

known_classes = sorted(train_df["label"].unique())
class_to_idx = {c:i for i,c in enumerate(known_classes)}
idx_to_class = {i:c for c,i in class_to_idx.items()}
print(class_to_idx)

class WaferMapDataset(Dataset):
    def __init__(self, df, class_to_idx=None, is_train=False, resize=64):
        self.df = df.reset_index(drop=True)
        self.class_to_idx = class_to_idx
        self.is_train = is_train
        self.resize = resize

    def __len__(self):
        return len(self.df)

    def _augment(self, x):
        # x: (1,H,W)
        if torch.rand(1).item() < 0.5:
            x = torch.flip(x, dims=[2])  # 좌우
        if torch.rand(1).item() < 0.5:
            x = torch.flip(x, dims=[1])  # 상하
        k = int(torch.randint(0, 4, (1,)).item())
        x = torch.rot90(x, k, dims=[1,2])
        return x

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        wm = row["waferMap"]  # numpy 2D
        x = torch.tensor(wm, dtype=torch.float32).unsqueeze(0)  # (1,H,W)

        # 0/1/2 형태면 0~1로 스케일 (선택)
        mx = x.max()
        x = x / (mx if mx > 0 else 1.0)

        # resize
        x = F.interpolate(x.unsqueeze(0), size=(self.resize, self.resize), mode="nearest").squeeze(0)

        if self.is_train:
            x = self._augment(x)

        # ResNet 입력 맞추기: 3채널
        x = x.repeat(3, 1, 1)  # (3,resize,resize)

        if self.class_to_idx is None:
            return x

        y = self.class_to_idx[row["label"]]
        return x, y

train_loader = DataLoader(WaferMapDataset(train_df, class_to_idx, True, 64),
                          batch_size=128, shuffle=True, num_workers=2)
val_loader = DataLoader(WaferMapDataset(val_df, class_to_idx, False, 64),
                        batch_size=256, shuffle=False, num_workers=2)
test_known_loader = DataLoader(WaferMapDataset(test_known_df, class_to_idx, False, 64),
                               batch_size=256, shuffle=False, num_workers=2)
test_unknown_loader = DataLoader(
    WaferMapDataset(test_unknown_df, class_to_idx=None, is_train=False, resize=64),
    batch_size=256, shuffle=False, num_workers=2
)

# Known 분류기 학습 (ResNet18 베이스라인)

In [ ]:
import torchvision.models as models
import torch.nn as nn
from tqdm import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"

model = models.resnet18(weights=None)
model.fc = nn.Linear(model.fc.in_features, len(known_classes))
model = model.to(device)

opt = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

def run_epoch(loader, train=True):
    model.train(train)
    total_loss, correct, total = 0, 0, 0
    for x, y in tqdm(loader, leave=False):
        x = x.to(device)
        y = torch.tensor(y).to(device)

        logits = model(x)
        loss = criterion(logits, y)

        if train:
            opt.zero_grad()
            loss.backward()
            opt.step()

        total_loss += loss.item() * x.size(0)
        pred = logits.argmax(dim=1)
        correct += (pred == y).sum().item()
        total += x.size(0)

    return total_loss/total, correct/total

for epoch in range(1, 6):
    tr_loss, tr_acc = run_epoch(train_loader, train=True)
    va_loss, va_acc = run_epoch(val_loader, train=False)
    print(f"Epoch {epoch} | train loss {tr_loss:.4f} acc {tr_acc:.4f} | val loss {va_loss:.4f} acc {va_acc:.4f}")

# 학습된 모델 저장하기

모델의 가중치(`state_dict`)를 저장합니다. 일반적으로 `.pth` 또는 `.pt` 확장자를 사용합니다.

In [ ]:
import os
import json
from datetime import datetime
import torch

# 1) 저장 폴더(Drive)
CKPT_DIR = "/content/drive/MyDrive/Colab Notebooks/WMI_Wafer_DefectPattern_Detection_AnomalyAnalysis/checkpoints"
os.makedirs(CKPT_DIR, exist_ok=True)

def _safe(s: str) -> str:
    # 파일명 안전하게(공백/슬래시 등 제거)
    return (
        str(s)
        .replace(" ", "")
        .replace("/", "-")
        .replace("\\", "-")
        .replace(",", "-")
        .replace("[", "")
        .replace("]", "")
        .replace("'", "")
        .replace('"', "")
    )

def make_run_name(
    arch: str,
    resize: int,
    unknown_classes,
    epochs: int = None,
    val_macro_f1: float = None,
    auroc_msp: float = None,
    auroc_energy: float = None,
):
    ts = datetime.now().strftime("%Y%m%d-%H%M%S")
    holdout = "-".join([_safe(x) for x in unknown_classes]) if unknown_classes else "none"
    parts = [
        ts,
        _safe(arch),
        f"r{resize}",
        f"holdout-{holdout}",
    ]
    if epochs is not None:
        parts.append(f"ep{int(epochs):02d}")
    if val_macro_f1 is not None:
        parts.append(f"valF1-{val_macro_f1:.3f}")
    if auroc_msp is not None:
        parts.append(f"aurocMSP-{auroc_msp:.3f}")
    if auroc_energy is not None:
        parts.append(f"aurocEn-{auroc_energy:.3f}")

    return "_".join(parts)

def save_checkpoint(
    path: str,
    model,
    class_to_idx: dict,
    unknown_classes: list,
    resize: int,
    arch: str = "resnet18",
    epochs: int = None,
    metrics: dict = None,
    extra_cfg: dict = None,
    optimizer=None,   # resume까지 원하면 전달
):
    ckpt = {
        "arch": arch,
        "resize": resize,
        "unknown_classes": unknown_classes,
        "class_to_idx": class_to_idx,
        "model_state_dict": model.state_dict(),
        "metrics": metrics or {},
        "cfg": extra_cfg or {},
        "saved_at": datetime.now().isoformat(),
    }
    if optimizer is not None:
        ckpt["optimizer_state_dict"] = optimizer.state_dict()

    torch.save(ckpt, path)
    print(f"✅ Saved checkpoint: {path}")
    # 사람이 보기 좋게 메타도 같이 저장(선택)
    meta_path = path.replace(".pt", ".json")
    with open(meta_path, "w", encoding="utf-8") as f:
        json.dump({k: ckpt[k] for k in ["arch","resize","unknown_classes","metrics","cfg","saved_at"]}, f, ensure_ascii=False, indent=2)
    print(f"📝 Saved meta: {meta_path}")


# Known 분류 성능을 “macro-F1”로 평가 (중요)

In [ ]:
import numpy as np
from sklearn.metrics import f1_score, confusion_matrix, classification_report

@torch.no_grad()
def predict_known(loader):
    model.eval()
    ys, ps = [], []
    for x, y in loader:
        x = x.to(device)
        logits = model(x)
        pred = logits.argmax(dim=1).cpu().numpy()
        ys.append(np.array(y))
        ps.append(pred)
    y_true = np.concatenate(ys)
    y_pred = np.concatenate(ps)
    return y_true, y_pred

y_true, y_pred = predict_known(test_known_loader)
print("macro-F1:", f1_score(y_true, y_pred, average="macro"))
print(classification_report(y_true, y_pred, target_names=[idx_to_class[i] for i in range(len(known_classes))]))

# Unknown 탐지(OOD): MSP vs Energy로 AUROC/AUPR 계산

In [ ]:
from sklearn.metrics import roc_auc_score, average_precision_score

@torch.no_grad()
def collect_scores(loader, known=True, T=1.0):
    model.eval()
    msp_list, energy_list = [], []
    for batch in loader:
        x = batch[0] if known else batch
        x = x.to(device)

        logits = model(x) / T
        prob = torch.softmax(logits, dim=1)
        msp = prob.max(dim=1).values                   # 높을수록 known
        energy = -T * torch.logsumexp(logits, dim=1)   # 보통 -energy가 높을수록 known로 사용

        msp_list.append(msp.cpu().numpy())
        energy_list.append(energy.cpu().numpy())

    return np.concatenate(msp_list), np.concatenate(energy_list)

msp_k, en_k = collect_scores(test_known_loader, known=True)
msp_u, en_u = collect_scores(test_unknown_loader, known=False)

y = np.concatenate([np.ones_like(msp_k), np.zeros_like(msp_u)])  # known=1, unknown=0

score_msp = np.concatenate([msp_k, msp_u])
score_energy = np.concatenate([-en_k, -en_u])  # -energy를 known 점수로

print("AUROC MSP   :", roc_auc_score(y, score_msp))
print("AUPR  MSP   :", average_precision_score(y, score_msp))
print("AUROC Energy:", roc_auc_score(y, score_energy))
print("AUPR  Energy:", average_precision_score(y, score_energy))

# 저장된 모델 불러오기

저장된 `state_dict`를 불러와 새로운 모델 인스턴스에 로드합니다. 이때, 모델의 아키텍처는 저장할 때와 동일하게 정의되어 있어야 합니다.

In [ ]:
import os, json
from datetime import datetime
import torch
import torchvision.models as models
import torch.nn as nn

CKPT_DIR = "/content/drive/MyDrive/Colab Notebooks/WMI_Wafer_DefectPattern_Detection_AnomalyAnalysis/checkpoints"
os.makedirs(CKPT_DIR, exist_ok=True)

def _safe(s: str) -> str:
    return (
        str(s).replace(" ", "")
        .replace("/", "-").replace("\\", "-")
        .replace(",", "-").replace("[", "").replace("]", "")
        .replace("'", "").replace('"', "")
    )

def _get(name, default=None):
    # 노트북 전역 변수에서 자동으로 가져오기
    return globals().get(name, default)

def save_mvp_checkpoint(
    model,
    class_to_idx,
    unknown_classes,
    resize=64,
    arch="resnet18",
    optimizer=None,
    ckpt_dir=CKPT_DIR,
    extra_cfg=None,
):
    """
    ✅ 자동 저장:
    - 노트북에 val_macro_f1 / auroc_msp / auroc_energy / epochs 등의 변수가 있으면 자동으로 메타+파일명에 반영
    - 없으면 없는대로 저장 (MVP 중간 저장에도 사용 가능)
    """
    os.makedirs(ckpt_dir, exist_ok=True)

    # 자동으로 메트릭/설정 값 끌어오기(있으면 쓰고, 없으면 None)
    epochs = _get("epochs_run", None) or _get("EPOCHS", None) or _get("epochs", None)
    val_macro_f1 = _get("val_macro_f1", None)
    auroc_msp = _get("auroc_msp", None)
    auroc_energy = _get("auroc_energy", None)
    holdout = "-".join([_safe(x) for x in (unknown_classes or [])]) or "none"
    ts = datetime.now().strftime("%Y%m%d-%H%M%S")

    # 파일명 구성(있는 정보만 추가)
    parts = [ts, _safe(arch), f"r{int(resize)}", f"holdout-{holdout}"]
    if epochs is not None:
        parts.append(f"ep{int(epochs):02d}")
    if val_macro_f1 is not None:
        parts.append(f"valF1-{float(val_macro_f1):.3f}")
    if auroc_msp is not None:
        parts.append(f"aurocMSP-{float(auroc_msp):.3f}")
    if auroc_energy is not None:
        parts.append(f"aurocEn-{float(auroc_energy):.3f}")

    run_name = "_".join(parts)
    ckpt_path = os.path.join(ckpt_dir, run_name + ".pt")

    # 체크포인트 내용
    ckpt = {
        "arch": arch,
        "resize": int(resize),
        "unknown_classes": list(unknown_classes) if unknown_classes is not None else None,
        "class_to_idx": dict(class_to_idx),
        "model_state_dict": model.state_dict(),
        "metrics": {
            "val_macro_f1": val_macro_f1,
            "auroc_msp": auroc_msp,
            "auroc_energy": auroc_energy,
        },
        "cfg": extra_cfg or {},
        "saved_at": datetime.now().isoformat(),
    }
    if optimizer is not None:
        ckpt["optimizer_state_dict"] = optimizer.state_dict()

    torch.save(ckpt, ckpt_path)

    # 사람이 보기 좋은 메타 json도 같이 저장(선택이지만 추천)
    meta_path = ckpt_path.replace(".pt", ".json")
    with open(meta_path, "w", encoding="utf-8") as f:
        json.dump(
            {k: ckpt[k] for k in ["arch", "resize", "unknown_classes", "metrics", "cfg", "saved_at"]},
            f, ensure_ascii=False, indent=2
        )

    print("✅ Saved:", ckpt_path)
    print("📝 Meta :", meta_path)
    return ckpt_path

def load_mvp_checkpoint(path, device="cpu"):
    ckpt = torch.load(path, map_location=device)

    class_to_idx = ckpt["class_to_idx"]
    known_classes = [None] * len(class_to_idx)
    for cls, idx in class_to_idx.items():
        known_classes[idx] = cls

    arch = ckpt.get("arch", "resnet18")
    if arch == "resnet18":
        model = models.resnet18(weights=None)
        model.fc = nn.Linear(model.fc.in_features, len(known_classes))
    else:
        raise ValueError(f"Unsupported arch: {arch}")

    model.load_state_dict(ckpt["model_state_dict"])
    model = model.to(device).eval()

    print("✅ Loaded:", path)
    print(" - arch:", arch, "| resize:", ckpt.get("resize"))
    print(" - unknown_classes:", ckpt.get("unknown_classes"))
    print(" - metrics:", ckpt.get("metrics", {}))
    return model, ckpt, known_classes, class_to_idx

In [ ]:
ckpt_path = save_mvp_checkpoint(model, class_to_idx, UNKNOWN_CLASSES, resize=64, arch="resnet18", optimizer=None)

In [ ]:
loaded_model, ckpt, known_classes_loaded, class_to_idx_loaded = load_mvp_checkpoint(ckpt_path, device=device)